# Tutorial: Combined Topic Modeling

(last updated 10-05-2021)

In this tutorial, we are going to use our **Combined Topic Model** to get the topics out of a collections of articles.

## Topic Models 

Topic models allow you to discover latent topics in your documents in a completely unsupervised way. Just use your documents and get topics out.

## Contextualized Topic Models

![](https://raw.githubusercontent.com/MilaNLProc/contextualized-topic-models/master/img/logo.png)

What are Contextualized Topic Models? **CTMs** are a family of topic models that combine the expressive power of BERT embeddings with the unsupervised capabilities of topic models to get topics out of documents. 

## Python Package

You can find our package [here](https://github.com/MilaNLProc/contextualized-topic-models).

![https://github.com/MilaNLProc/contextualized-topic-models/actions](https://github.com/MilaNLProc/contextualized-topic-models/workflows/Python%20package/badge.svg) ![https://pypi.python.org/pypi/contextualized_topic_models](https://img.shields.io/pypi/v/contextualized_topic_models.svg) ![https://pepy.tech/badge/contextualized-topic-models](https://pepy.tech/badge/contextualized-topic-models)

# **Before you start...**

If you have additional questions about these topics, follow the links:

- you need to work with languages different than English: [click here!](https://contextualized-topic-models.readthedocs.io/en/latest/language.html#language-specific)
- you can't get good results with topic models: [click here!](https://contextualized-topic-models.readthedocs.io/en/latest/faq.html#i-am-getting-very-poor-results-what-can-i-do)
- you want to load your own embeddings: [click here!](https://contextualized-topic-models.readthedocs.io/en/latest/faq.html#can-i-load-my-own-embeddings)


# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing Contextualized Topic Models

First, we install the contextualized topic model library

In [ ]:
%%capture
!pip install contextualized-topic-models==2.2.0

In [ ]:
%%capture
!pip install pyldavis

## Restart the Notebook

For the changes to take effect, we now need to restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data

We are going to need some data. You should upload a file with one document per line. We assume you haven't run any preprocessing script.

However, if you want to first test the model without uploading your data, you can simply use the test file I'm putting here

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

In [ ]:
!head -n 1 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry


In [ ]:
text_file = "dbpedia_sample_abstract_20k_unprep.txt" # EDIT THIS WITH THE FILE YOU UPLOAD

# Importing what we need

In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk

## Preprocessing

Why do we use the **preprocessed text** here? We need text without punctuation to build the bag of word. Also, we might want only to have the most frequent words inside the BoW. Too many words might not help.

In [ ]:
nltk.download('stopwords')

documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]
sp = WhiteSpacePreprocessing(documents, stopwords_language='english')
preprocessed_documents, unpreprocessed_corpus, vocab = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
preprocessed_documents[:2]

['mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry',
 'died march american photographer specialized photography operated studio silver spring maryland later lived florida magazine photographer year']

We don't discard the non-preprocessed texts, because we are going to use them as input for obtaining the contextualized document representations. 

Let's pass our files with preprocess and unpreprocessed data to our `TopicModelDataPreparation` object. This object takes care of creating the bag of words for you and of obtaining the contextualized BERT representations of documents. This operation allows us to create our training dataset.

Note: Here we use the contextualized model "paraphrase-distilroberta-base-v1".


In [ ]:
tp = TopicModelDataPreparation("paraphrase-distilroberta-base-v1")

training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

  0%|          | 0.00/306M [00:00<?, ?B/s]

Batches:   0%|          | 0/100 [00:00<?, ?it/s]

Let's check the first ten words of the vocabulary 

In [ ]:
tp.vocab[:10]

['abbreviated',
 'academic',
 'academy',
 'access',
 'according',
 'achieved',
 'acquired',
 'acre',
 'acres',
 'across']

## Training our Combined TM

Finally, we can fit our new topic model. We will ask the model to find 50 topics in our collection.

In [ ]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset) # run the model

Epoch: [20/20]	 Seen Samples: [400000/400000]	Train Loss: 135.8926830078125	Time: 0:00:05.510567: : 20it [01:49,  5.49s/it]


# Topics

After training, now it is the time to look at our topics: we can use the 

```
get_topic_lists
```

function to get the topics. It also accepts a parameter that allows you to select how many words you want to see for each topic.

If you look at the topics, you will see that they all make sense and are representative of a collection of documents that comes from Wikipedia (general knowledge). Notice that the topics are in English, because we trained the model on English documents.

In [ ]:
ctm.get_topic_lists(5)

[['mi', 'kilometres', 'village', 'north', 'municipality'],
 ['national', 'house', 'built', 'historic', 'county'],
 ['used', 'defined', 'mathematics', 'number', 'typically'],
 ['film', 'directed', 'best', 'films', 'produced'],
 ['united', 'states', 'company', 'air', 'international'],
 ['species', 'family', 'found', 'native', 'genus'],
 ['composer', 'january', 'painter', 'studied', 'son'],
 ['station', 'line', 'located', 'railway', 'street'],
 ['church', 'roman', 'century', 'ancient', 'catholic'],
 ['county', 'school', 'high', 'state', 'located'],
 ['published', 'book', 'work', 'books', 'writer'],
 ['held', 'season', 'tournament', 'cup', 'championship'],
 ['made', 'born', 'english', 'first', 'played'],
 ['built', 'house', 'story', 'building', 'style'],
 ['war', 'world', 'series', 'television', 'first'],
 ['government', 'political', 'party', 'council', 'act'],
 ['american', 'new', 'york', 'known', 'born'],
 ['school', 'high', 'college', 'located', 'secondary'],
 ['published', 'written', '

# Let's Draw!

We can use PyLDAvis to plot our topic in a nice and friendly manner :)

In [ ]:
 lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

Sampling: [10/10]: : 10it [00:49,  4.99s/it]


In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

# Topic Predictions

Ok now we can take a document and see which topic has been assigned to it. Results will obviously change with respect to the documents you are using. For example, let's predict the topic of the first preprocessed document that is talking about a peninsula.

In [ ]:
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5) # get all the topic predictions

Sampling: [5/5]: : 5it [00:24,  4.97s/it]


In [ ]:
preprocessed_documents[0] # see the text of our preprocessed document

'mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry'

In [ ]:
import numpy as np
topic_number = np.argmax(topics_predictions[0]) # get the topic id of the first document

In [ ]:
ctm.get_topic_lists(5)[topic_number] #and the topic should be about natural location related things

['located', 'park', 'city', 'state', 'county']

# Save Our Model for Later Use

In [ ]:
ctm.save(models_dir="./")

/content/contextualized-topic-models/contextualized_topic_models/models/ctm.py:446: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  Warning)


In [ ]:
# let's remove the trained model
del ctm

In [ ]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, num_epochs=100, n_components=50)

ctm.load("/content/contextualized_topic_model_nc_50_tpm_0.0_tpv_0.98_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99",
                                                                                                      epoch=19)

/content/contextualized-topic-models/contextualized_topic_models/models/ctm.py:471: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  Warning)


In [ ]:
ctm.get_topic_lists(5)

[['mi', 'east', 'lies', 'south', 'village'],
 ['world', 'women', 'team', 'cup', 'international'],
 ['founded', 'school', 'education', 'university', 'established'],
 ['studied', 'painter', 'paris', 'french', 'german'],
 ['played', 'born', 'english', 'made', 'right'],
 ['film', 'directed', 'written', 'produced', 'stars'],
 ['league', 'football', 'played', 'american', 'team'],
 ['album', 'released', 'island', 'river', 'band'],
 ['de', 'greek', 'king', 'french', 'son'],
 ['member', 'party', 'politician', 'general', 'elected'],
 ['university', 'american', 'professor', 'received', 'college'],
 ['church', 'roman', 'catholic', 'century', 'diocese'],
 ['municipality', 'region', 'area', 'kilometres', 'mi'],
 ['mi', 'west', 'km', 'county', 'south'],
 ['population', 'area', 'county', 'town', 'census'],
 ['built', 'house', 'building', 'story', 'style'],
 ['game', 'developed', 'video', 'games', 'playstation'],
 ['used', 'often', 'term', 'form', 'usually'],
 ['published', 'book', 'written', 'books', 